In [2]:
## MLFLOW_TRACKING_URI='https://qualiextra-project-6b29e931dd67.herokuapp.com/'

In [1]:
import pandas as pd
import datetime


In [2]:
data = pd.read_csv("./Missions.csv", sep=';')

In [3]:
def load_data():
        data = pd.read_csv("./Missions.csv", sep=";")
        mask = data["hôtel"].notna()
        data = data[mask]
        data["date_debut"] = data.apply(lambda x: x["date"].split(" →")[0], axis=1)
        data["date_debut"] = data["date_debut"].apply(
            lambda x: x.replace(" (UTC+3)", "")
        )
        data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC)", ""))
        data["date_debut"] = pd.to_datetime(data["date_debut"], format="%d/%m/%Y %H:%M")
        data["time_delta"] = data["nbre d'heures"].apply(lambda x: pd.to_timedelta(x))
        data["date_fin"] = data.apply(
            lambda x: x["date_debut"] + x["time_delta"], axis=1
        )
        data["Propriété"] = data.apply(
            lambda x: x["hôtel"].split(" (")[0] if "www" in x["Propriété"] else x,
            axis=1,
        )["Propriété"]
        data["Propriété_clean"] = data["hôtel"].apply(lambda x: x.split(" (")[0])
        data["extra_clean"] = data["extra"].apply(lambda x: x.split(" (")[0])
        data["periode_debut"] = data["date_debut"].dt.strftime("%m-%Y")
        data["periode_fin"] = data["date_fin"].dt.strftime("%m-%Y")
        data["Année"] = data["date_fin"].dt.year.astype(str)
        data["Mois"] = data["date_fin"].dt.to_period("M").astype(str)
        data["Semaine"] = data["date_fin"].dt.to_period("W-Mon").astype(str)
        data["marge"] = data.apply(lambda x: x["total HT"] - x["montant HT"], axis=1)
        return data

In [4]:
data = load_data()

In [12]:
data[data["Année"] == "2023"].groupby("Mois")["extra_clean"].unique().count()

Mois
2023-01    [Serine Adili , Mohamed Dendani, Yannel Tchiss...
2023-02    [Yannel Tchissambou , Mamadou Koma, Léopold Me...
2023-03    [Rachel Magtoto, Yosra Saai, Alexandra Patel, ...
2023-04    [Yosra Saai, Sorel Pambou, Mamadou Koma, Antho...
2023-05    [Shaines Ait Lhaj, Yosra Saai, Emma Badani, OK...
2023-06    [Shaines Ait Lhaj, Alain Itoua, Chabane Yahi, ...
2023-07    [Anthony Justin, Grégory Tshimini, Haissam Bag...
2023-08    [Shaines Ait Lhaj, Alain Itoua, Serine Adili ,...
2023-09    [Alain Itoua, Léopold Mendy, Yacine Diallo, Je...
2023-10    [OKALA ESSONO Isilda Cathy, Rym Bouabdallah, H...
2023-11    [Léopold Mendy, Mourad Riffi, Mamadou Koma, Al...
2023-12    [Alain Itoua, Mamadou Koma, Haissam Baghdadi, ...
Name: extra_clean, dtype: object

In [11]:
data[data["Année"] == "2023"].groupby("Mois").count()

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,...,date_debut,time_delta,date_fin,Propriété_clean,extra_clean,periode_debut,periode_fin,Année,Semaine,marge
Mois,,,,,,,,,,,,,,,,,,,,,
2023-01,221,221,221,221,49,16,221,221,221,188,...,221,221,221,221,221,221,221,221,221,221
2023-02,215,215,215,215,39,8,215,215,215,184,...,215,215,215,215,215,215,215,215,215,215
2023-03,213,213,213,213,13,3,213,213,213,203,...,213,213,213,213,213,213,213,213,213,213
2023-04,208,208,208,208,25,10,208,208,208,193,...,208,208,208,208,208,208,208,208,208,208
2023-05,203,203,203,203,17,11,203,203,203,193,...,203,203,203,203,203,203,203,203,203,203
2023-06,255,255,255,255,26,7,255,255,255,234,...,255,255,255,255,255,255,255,255,255,255
2023-07,305,305,305,305,38,5,305,305,305,273,...,305,305,305,305,305,305,305,305,305,305
2023-08,335,335,335,335,50,7,335,335,335,287,...,335,335,335,335,335,335,335,335,335,335
2023-09,290,290,290,290,31,12,290,290,290,276,...,290,290,290,290,290,290,290,290,290,290


In [ ]:
fig = go.Figure()

for period in data["Année"].unique():
    fig.add_trace(
        go.Scatter(
            x=data[data["Année"] == period].groupby(),
            y=data[data["Année"] == period].groupby("Mois").count()["extra_clean"],
        )
    )

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4771 entries, 0 to 4770
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Propriété      4764 non-null   object 
 1   hôtel          4760 non-null   object 
 2   extra          4761 non-null   object 
 3   date           4771 non-null   object 
 4   statuts        617 non-null    object 
 5   tarif urgence  225 non-null    float64
 6   tarif horaire  4760 non-null   float64
 7   nbre d'heures  4771 non-null   object 
 8   total HT       4771 non-null   int64  
 9   facture        3798 non-null   object 
 10  règlement      3743 non-null   object 
 11  montant HT     4771 non-null   int64  
 12  Texte          7 non-null      object 
dtypes: float64(2), int64(2), object(9)
memory usage: 484.7+ KB


In [6]:
mask = ((data["hôtel"].notna()))
data = data[mask]
data.shape

(4760, 13)

In [7]:
data["date_debut"] = data.apply(lambda x: x["date"].split(" →")[0],axis=1)
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC+3)", ""))
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC)", ""))

In [8]:
data["date_debut"] = pd.to_datetime(data["date_debut"], format="%d/%m/%Y %H:%M")

In [9]:
data.dtypes

Propriété                object
hôtel                    object
extra                    object
date                     object
statuts                  object
tarif urgence           float64
tarif horaire           float64
nbre d'heures            object
total HT                  int64
facture                  object
règlement                object
montant HT                int64
Texte                    object
date_debut       datetime64[ns]
dtype: object

In [10]:
data["time_delta"] = data["nbre d'heures"].apply(lambda x: pd.to_timedelta(x))
data["date_fin"] = data.apply(lambda x: x["date_debut"] + x["time_delta"], axis=1)
data["Propriété"] = data.apply(lambda x: x["hôtel"].split(" (")[0] if "www" in x["Propriété"] else x, axis=1)["Propriété"]
data["Propriété_clean"] = data["hôtel"].apply(lambda x: x.split(" (")[0])
data["extra_clean"] = data["extra"].apply(lambda x: x.split(" (")[0])
data["periode_debut"] = data["date_debut"].dt.strftime('%m-%Y')
data.head()

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,règlement,montant HT,Texte,date_debut,time_delta,date_fin,Propriété_clean,extra_clean,periode_debut
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,07/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-07 08:00:00,0 days 12:00:00,2024-01-07 20:00:00,Crystal,Grégory Tshimini,01-2024
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,06/01/2024 22:30 → 07/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-06 22:30:00,0 days 09:30:00,2024-01-07 08:00:00,Milie Rose,Mamadou Koma,01-2024
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,06/01/2024 19:30 → 07/01/2024 7:30,NaN,NaN,20.0,12hr 0min,240,NaN,NaN,180,NaN,2024-01-06 19:30:00,0 days 12:00:00,2024-01-07 07:30:00,Touraine,Mohamed Belbachir,01-2024
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,06/01/2024 8:00 → 20:00,NaN,NaN,21.0,12hr 0min,252,NaN,NaN,180,NaN,2024-01-06 08:00:00,0 days 12:00:00,2024-01-06 20:00:00,Crystal,Grégory Tshimini,01-2024
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,05/01/2024 22:30 → 06/01/2024 8:00,NaN,NaN,20.0,9hr 30min,180,NaN,NaN,135,NaN,2024-01-05 22:30:00,0 days 09:30:00,2024-01-06 08:00:00,Milie Rose,Mamadou Koma,01-2024


In [20]:
import plotly.graph_objects as go 

fig = go.Figure()

fig.add_trace(go.Histogram(x = data['periode_debut'], y= data['extra_clean']))
fig.update_traces(dict(marker_line_width=0))
months = data['periode_debut'].to_list()
buttons = [
    go.layout.updatemenu.Button(
        label=month,
        method='update',
        args=[{'visible': [m == month for m in months]}]
    )
    for month in months
]
fig.show()

In [10]:
data["date_debut"].dt.year.value_counts()

date_debut
2023    2827
2022    1841
2021      82
2024      10
Name: count, dtype: int64

In [13]:
import plotly.express as px

px.histogram(data["date_debut"].dt.year.astype(str))